# Indonesian License Plate Recognition & Region Identification

This notebook demonstrates how to use the `fast-alpr` library to perform Automatic License Plate Recognition (ALPR) on images, with a special focus on processing and understanding Indonesian license plates.

**We will cover two main parts:**
1.  **Basic Usage**: A quick look at how to get instant predictions and annotated images with just a few lines of code.
2.  **Advanced Processing**: A more detailed approach where we'll:
    * Extract the license plate text.
    * Validate and format it according to Indonesian standards.
    * Identify the plate's region of origin (e.g., `D` for Bandung Raya).
    * Display the structured results and custom annotations.

**Libraries Used:**
* `fast-alpr`: The core engine for detection and OCR.
* `opencv-python`: For image loading and manipulation.
* `matplotlib`: For displaying images neatly within the notebook.


## 1. Setup and Installation

First, let's make sure we have all the necessary libraries installed. If you haven't installed `fast-alpr` or the other dependencies, you can run the following command.

In [ ]:
# %%capture
# The '%%capture' magic command suppresses the output of this cell.
# Uncomment and run the line below if you need to install the libraries.
# !pip install fast-alpr opencv-python matplotlib

## 2. Imports and Global Configurations

Here, we import the required libraries and set up our global configurations, including the path to our test image and the dictionary for Indonesian license plate region codes.

**Important:** Please update the `image_path` variable to point to the location of your image file.


In [1]:
import cv2
import re
from pathlib import Path
import matplotlib.pyplot as plt

# --- Third-Party Imports ---
from fast_alpr import ALPR

# --- Configuration ---

# UPDATE THIS PATH to your image file.
# Using Path() makes the code work on any OS (Windows, macOS, Linux).
image_path = Path("D:/YOLO/3-6-25 (1).jpg") # <--- CHANGE THIS

# --- Indonesian City Code Dictionary ---
# A simplified dictionary mapping license plate prefixes to their corresponding regions.
# For a more detailed version, this dictionary could be expanded or loaded from a file.
cityCode_dict = {
    "AA": ["Purworejo", "Temanggung", "Magelang", "Wonosobo", "Kebumen"],
    "B":  ["DKI Jakarta", "Bekasi", "Depok", "Tangerang"],
    "D":  ["Bandung", "Bandung Barat", "Cimahi"],
    "AG": ["Tulungagung", "Kediri", "Blitar", "Trenggalek", "Nganjuk"],
    "BA": "Sumatera Barat",
    "BP": "Kepulauan Riau",
    "DK": "Bali",
    "H":  "Semarang",
    "L":  "Surabaya",
    "M":  "Madura",
    "N":  ["Malang", "Pasuruan", "Probolinggo", "Batu", "Lumajang"],
    "PA": "Papua",
    "PB": "Papua Barat",
    "S":  ["Tuban", "Jombang", "Bojonegoro", "Lamongan", "Mojokerto"],
    "W":  ["Gresik", "Sidoarjo"],
}

# --- Matplotlib Display Helper ---
# A small helper function to display images correctly in the notebook.
def display_image(image, title=""):
    """Displays an OpenCV image (BGR) correctly in a Matplotlib plot."""
    # Convert from BGR (OpenCV's default) to RGB (Matplotlib's default)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(12, 8))
    plt.imshow(rgb_image)
    plt.title(title, fontsize=16)
    plt.axis('off') # Hide the axes
    plt.show()


## 3. Part 1: Quick & Simple ALPR

This section demonstrates the most straightforward way to use `fast-alpr`. We initialize the `ALPR` class and use its built-in `draw_predictions` method to get an annotated image instantly. This is great for quick visualization without needing to process the raw data.


In [ ]:
# --- Step 1: Initialize the ALPR model ---
# We specify the detection and OCR models to use.
# These models are downloaded automatically on the first run.
alpr_basic = ALPR(
    detector_model="yolo-v9-s-608-license-plate-end2end",
    ocr_model="global-plates-mobile-vit-v2-model",
)

# --- Step 2: Load the image ---
# Check if the image file exists before trying to load it.
if not image_path.is_file():
    print(f"❌ Error: Image file not found at '{image_path}'")
    print("Please update the 'image_path' variable in the cell above.")
else:
    frame = cv2.imread(str(image_path))

    # --- Step 3: Get and Draw Predictions ---
    # This single method handles both detection and drawing the results.
    annotated_frame = alpr_basic.draw_predictions(frame)

    # --- Step 4: Display the Result ---
    print(f"✅ Basic ALPR result for: {image_path.name}")
    display_image(annotated_frame, title="Basic ALPR Result")

    # --- Step 5: Save the Annotated Image ---
    # We create a new filename by appending '_basic_OCR' to the original stem.
    output_path = image_path.with_stem(image_path.stem + "_basic_OCR").with_suffix(".jpg")
    cv2.imwrite(str(output_path), annotated_frame)
    print(f"💾 Annotated image saved to: {output_path}")


## 4. Part 2: Advanced Processing for Indonesian Plates

While the basic method is fast, we often need more control. In this part, we'll get the raw prediction data and process it ourselves. This allows us to:
1.  Clean and validate the OCR text.
2.  Extract the region code.
3.  Look up the region information from our dictionary.
4.  Print a structured summary of the findings.
5.  Draw our own custom annotations on the image.

### Helper Functions
First, let's define the functions that will help us process the ALPR results.


In [ ]:
def extract_indonesian_plate_format(text: str) -> tuple[str | None, str | None]:
    """
    Validates and extracts components from OCR text to match Indonesian license plate formats.

    Args:
        text (str): The raw OCR text from the license plate.

    Returns:
        tuple[str | None, str | None]: A tuple containing the formatted plate string
                                       and the region code, or (None, None) if no valid
                                       pattern is matched.
    """
    # First, clean the input text by removing all whitespace and converting to uppercase.
    clean_text = re.sub(r'\s+', '', text.upper())

    # Define regex patterns for Indonesian plates.
    # The pattern is flexible: 1-2 letters, 1-4 numbers, and 1-3 letters.
    # This covers most common plate variations (e.g., D 1234 ABC, B 123 CD).
    pattern = r'^([A-Z]{1,2})(\d{1,4})([A-Z]{1,3})$'

    match = re.match(pattern, clean_text)
    if match:
        region_code = match.group(1)
        numbers = match.group(2)
        suffix = match.group(3)
        # Reconstruct the plate into a standardized format.
        formatted_plate = f"{region_code} {numbers} {suffix}"
        return formatted_plate, region_code

    # If no pattern matches, return None.
    return None, None

def get_region_info(region_code: str) -> str:
    """
    Looks up the region name(s) from the cityCode_dict based on the region code.

    Args:
        region_code (str): The prefix of the license plate (e.g., "D", "B").

    Returns:
        str: A formatted string containing the region information.
    """
    if region_code in cityCode_dict:
        regions = cityCode_dict[region_code]
        # Handle cases where a code maps to multiple cities (a list).
        if isinstance(regions, list):
            return f"{', '.join(regions)}"
        # Handle cases where a code maps to a single city (a string).
        else:
            return f"{regions}"
    else:
        return "Region not found in database"

def process_alpr_results(alpr_instance: ALPR, frame: "np.ndarray") -> list[dict]:
    """
    Processes a frame using ALPR to find and structure data for Indonesian plates.

    Args:
        alpr_instance (ALPR): The initialized ALPR model.
        frame (np.ndarray): The image frame to process.

    Returns:
        list[dict]: A list of dictionaries, where each dictionary contains
                    structured information about a detected Indonesian plate.
    """
    # Get the raw prediction data from the ALPR model.
    predictions = alpr_instance.predict(frame)
    detected_plates = []

    for prediction in predictions:
        # Extract the OCR text.
        ocr_text = prediction.ocr.text
        formatted_plate, region_code = extract_indonesian_plate_format(ocr_text)

        # If the text matches a valid Indonesian format, process it further.
        if formatted_plate and region_code:
            region_info = get_region_info(region_code)
            
            # Store all relevant information in a dictionary.
            plate_info = {
                'original_text': ocr_text,
                'formatted_plate': formatted_plate,
                'region_code': region_code,
                'region_info': region_info,
                'confidence': prediction.ocr.confidence,
                'bbox': prediction.bbox_xyxy # Bounding box coordinates [x1, y1, x2, y2]
            }
            detected_plates.append(plate_info)

    return detected_plates


### Main Processing and Visualization
Now, let's use the helper functions to perform the advanced analysis.



In [ ]:
# --- Step 1: Initialize the ALPR model (can be a new instance or reuse the old one) ---
alpr_advanced = ALPR(
    detector_model="yolo-v9-s-608-license-plate-end2end",
    ocr_model="global-plates-mobile-vit-v2-model",
)

# --- Step 2: Load the image ---
if not image_path.is_file():
    print(f"❌ Error: Image file not found at '{image_path}'")
else:
    frame = cv2.imread(str(image_path))
    
    # --- Step 3: Process the image to get structured plate data ---
    detected_plates = process_alpr_results(alpr_advanced, frame)

    # --- Step 4: Draw custom annotations on the frame ---
    # We'll work on a copy to keep the original frame clean.
    annotated_frame_advanced = frame.copy()

    for plate in detected_plates:
        # Get the bounding box coordinates.
        x1, y1, x2, y2 = map(int, plate['bbox'])
        
        # Define the label text with the formatted plate and region info.
        label = f"{plate['formatted_plate']} ({plate['region_info']})"
        
        # Draw the bounding box.
        cv2.rectangle(annotated_frame_advanced, (x1, y1), (x2, y2), (0, 255, 0), 3) # Green box
        
        # Draw a filled background for the text for better visibility.
        cv2.rectangle(annotated_frame_advanced, (x1, y1 - 30), (x2, y1), (0, 255, 0), -1)
        
        # Put the custom label text above the bounding box.
        cv2.putText(
            annotated_frame_advanced, label, (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2 # Black text
        )

    # --- Step 5: Display the custom annotated image ---
    print("✅ Advanced ALPR processing complete.")
    display_image(annotated_frame_advanced, title="Advanced Indonesian ALPR Result")
    
    # --- Step 6: Save the new annotated image ---
    output_path_advanced = image_path.with_stem(image_path.stem + "_advanced_ALPR").with_suffix(".jpg")
    cv2.imwrite(str(output_path_advanced), annotated_frame_advanced)
    print(f"💾 Advanced annotated image saved to: {output_path_advanced}")


### Structured Results
Finally, let's print the structured data we extracted. This kind of output is perfect for saving to a CSV file, a database, or for further analysis.


In [ ]:
# Print the structured results in a clean, readable format.
print("=" * 60)
print("🇮🇩 INDONESIAN LICENSE PLATE DETECTION RESULTS 🇮🇩")
print("=" * 60)

if detected_plates:
    for i, plate in enumerate(detected_plates, 1):
        print(f"\n प्लेट {i} ({plate['region_info']}):") # Plate in Hindi
        print(f"  ┃")
        print(f"  ┣━ Original OCR Text: '{plate['original_text']}'")
        print(f"  ┣━ Formatted Plate:   '{plate['formatted_plate']}'")
        print(f"  ┗━ Confidence:        {plate['confidence']:.2%}") # Display as percentage
    print("-" * 60)
else:
    print("No valid Indonesian license plates were detected in the image.")

print(f"\nTotal valid plates detected: {len(detected_plates)}")
print("=" * 60)
